# Cloud Workshop Databricks
## 10. Déploiement de modèle avec Azure ML Service

## 1. Accès aux données

In [3]:
import uuid
import os

tempFolderName = '/FileStore/aml_labs_{0}'.format(uuid.uuid4())
dbutils.fs.mkdirs(tempFolderName)
print('Données sauvergardées ici : {0}'.format(tempFolderName))

filesToDownload = ['UsedCars_Clean.csv', 'UsedCars_Affordability.csv']

for fileToDownload in filesToDownload:
  downloadCommand = 'wget -O ''/dbfs{0}/{1}'' ''https://databricksdemostore.blob.core.windows.net/data/aml-labs/{1}'''.format(tempFolderName, fileToDownload)
  print(downloadCommand)
  os.system(downloadCommand)
  
dbutils.fs.ls(tempFolderName)

Données sauvergardées ici : /FileStore/aml_labs_9286bb11-1e28-4688-b498-fba2cd6b8fc2
wget -O /dbfs/FileStore/aml_labs_9286bb11-1e28-4688-b498-fba2cd6b8fc2/UsedCars_Clean.csv https://databricksdemostore.blob.core.windows.net/data/aml-labs/UsedCars_Clean.csv
wget -O /dbfs/FileStore/aml_labs_9286bb11-1e28-4688-b498-fba2cd6b8fc2/UsedCars_Affordability.csv https://databricksdemostore.blob.core.windows.net/data/aml-labs/UsedCars_Affordability.csv
 Out[ 1 ]: 
[FileInfo(path='dbfs:/FileStore/aml_labs_9286bb11-1e28-4688-b498-fba2cd6b8fc2/UsedCars_Affordability.csv', name='UsedCars_Affordability.csv', size=0),
 FileInfo(path='dbfs:/FileStore/aml_labs_9286bb11-1e28-4688-b498-fba2cd6b8fc2/UsedCars_Clean.csv', name='UsedCars_Clean.csv', size=60530)]

## 2. Estimation d'un modèle de Machine Learning

In [5]:
import os
import numpy as np
import pandas as pd
from sklearn import linear_model 
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import azureml
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
from azureml.core.model import Model 
import pickle
import json

print("Version Azure ML Service :", azureml.core.VERSION)
print()

pathToCsvFile = os.path.join('/dbfs' + tempFolderName, 'UsedCars_Affordability.csv')
df_affordability = pd.read_csv(pathToCsvFile, delimiter=',')
print(df_affordability)

full_X = df_affordability[["Age", "KM"]]
full_Y = df_affordability[["Affordable"]]

def train_eval_register_model(ws, experiment_name, model_name, full_X, full_Y,training_set_percentage):

    myexperiment = Experiment(ws, experiment_name)
    run = myexperiment.start_logging()

    train_X, test_X, train_Y, test_Y = train_test_split(full_X, full_Y, train_size=training_set_percentage, random_state=42)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(train_X)
    clf = linear_model.LogisticRegression(C=1)
    clf.fit(X_scaled, train_Y)

    scaled_inputs = scaler.transform(test_X)
    predictions = clf.predict(scaled_inputs)
    score = accuracy_score(test_Y, predictions)

    print("With %0.2f percent of data, model accuracy reached %0.4f." % (training_set_percentage, score))

    # Log the training metrics to Azure Machine Learning service run history
    run.log("Training_Set_Percentage", training_set_percentage)
    run.log("Accuracy", score)

    # Serialize the model to a pickle file in the outputs folder
    output_model_path = 'outputs/' + model_name + '.pkl'
    pickle.dump(clf,open(output_model_path,'wb'))
    print('Exported model to ', output_model_path)

    # Serialize the scaler as a pickle file in the same folder as the model
    output_scaler_path = 'outputs/' + 'scaler' + '.pkl'
    pickle.dump(scaler,open(output_scaler_path,'wb'))
    print('Exported scaler to ', output_scaler_path)

    # notice for the model_path, we supply the name of the outputs folder without a trailing slash
    # this will ensure both the model and the scaler get uploaded.
    registered_model = Model.register(model_path='outputs', model_name=model_name, workspace=ws)

    print(registered_model.name, registered_model.id, registered_model.version, sep = '\t')

    run.complete()

    return (registered_model, clf, scaler, score, run)

Version Azure ML Service : 1.0.18

 Age KM Affordable
0 23 46986 0
1 23 72937 0
2 24 41711 0
3 26 48000 0
4 30 38500 0
5 32 61000 0
6 27 94612 0
7 30 75889 0
8 27 19700 0
9 23 71138 0
10 25 31461 0
11 22 43610 0
12 25 32189 0
13 31 23000 0
14 32 34131 0
15 28 18739 0
16 30 34000 0
17 24 21716 0
18 24 25563 0
19 30 64359 0
20 30 67660 0
21 29 43905 0
22 28 56349 0
23 28 32220 0
24 29 25813 0
25 25 28450 0
26 27 34545 0
27 29 41415 0
28 28 44142 0
29 30 11090 0
... ... ... ...
1406 70 44850 1
1407 69 44826 1
1408 80 44444 1
1409 75 43720 1
1410 78 43622 1
1411 76 43532 1
1412 69 42800 1
1413 74 42317 1
1414 80 42186 1
1415 72 42000 1
1416 79 40093 1
1417 79 39800 1
1418 73 39168 1
1419 75 38945 1
1420 76 36537 1
1421 78 36000 1
1422 78 36000 1
1423 80 35821 1
1424 73 34717 1
1425 80 34000 1
1426 78 30964 1
1427 71 29000 1
1428 72 26000 1
1429 78 24000 1
1430 80 23000 1
1431 69 20544 1
1432 72 19000 1
1433 71 17016 1
1434 70 16916 1
1435 76 1 1

[1436 rows x 3 columns]

## 2.1 Accès à Azure ML service

In [7]:
subscription_id = "A COMPLETER" #you should be owner or contributor
resource_group = "A COMPLETER" #you should be owner or contributor
workspace_name = "A COMPLETER" #your workspace name
workspace_region = "West Europe" #your region

In [8]:
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

print("Provisionnement du workspace Azure ML Service : OK")
print()

experiment_name = "cloudworkshopADB"
model_name = "usedcarsmodel"
training_set_percentage = 0.50
registered_model, model, scaler, score, run = train_eval_register_model(ws, experiment_name, model_name, full_X, full_Y, training_set_percentage)

Provisionnement du workspace Azure ML Service : OK

With 0.50 percent of data, model accuracy reached 0.9109.
Exported model to outputs/usedcarsmodel.pkl
Exported scaler to outputs/scaler.pkl
Registering model usedcarsmodel
usedcarsmodel	usedcarsmodel:18	18

## 3. Récupération du modèle pour le scoring

In [10]:
# Download the model to a local directory
model_path = Model.get_model_path(model_name, _workspace=ws)
print('Model download to: ' + model_path)
age = 60
km = 40000

# Re-load the model
scaler = pickle.load(open(os.path.join(model_path,'scaler.pkl'),'rb'))
scaled_input = scaler.transform([[age, km]])
model2 = pickle.load(open(os.path.join(model_path,'usedcarsmodel.pkl'), 'rb'))


Model download to: azureml-models/usedcarsmodel/18/outputs
Scoring : [1]

Scoring : [1]

## 4. Création Azure Container Instance (ACI)

In [12]:
from azureml.core.conda_dependencies import CondaDependencies 

mycondaenv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas'])

with open("mydeployenv.yml","w") as f:
    f.write(mycondaenv.serialize_to_string())

In [13]:
#%%writefile score.py
scoring_service = """
import json
import os
import numpy as np
import pandas as pd
from sklearn import linear_model 
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import pickle
from sklearn.externals import joblib

def init():
    try:
        # One-time initialization of predictive model and scaler
        from azureml.core.model import Model
        
        global trainedModel   
        global scaler

        model_name = "usedcarsmodel" 
        model_path = Model.get_model_path(model_name)
        print('Looking for models in: ', model_path)

        trainedModel = pickle.load(open(os.path.join(model_path,'usedcarsmodel.pkl'), 'rb'))
        
        scaler = pickle.load(open(os.path.join(model_path,'scaler.pkl'),'rb'))

    except Exception as e:
        print('Exception during init: ', str(e))

def run(input_json):     
    try:
        inputs = json.loads(input_json)

        #Scale the input
        scaled_input = scaler.transform(inputs)
        
        #Get the scored result
        prediction = json.dumps(trainedModel.predict(scaled_input).tolist())

    except Exception as e:
        prediction = str(e)
    return prediction
""" 

with open("score.py", "w") as file:
    file.write(scoring_service)

## 4.1 Définition de l'image Container

In [15]:
runtime = "python" 
driver_file = "score.py"
conda_file = "mydeployenv.yml"

from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = driver_file,
                                                  runtime = runtime,
                                                  conda_file = conda_file)

## 5. Déploiement image dans Azure Container Instance (ACI)

Azure Container Instances propose la façon la plus simple et rapide d’exécuter un conteneur dans Azure, sans avoir à configurer des machines virtuelles et sans avoir à adopter un service de niveau supérieur. Découvrez comment créer et gérer des instances de conteneurs avec nos démarrages rapides, nos didacticiels et nos exemples.

> Plus d'info : https://azure.microsoft.com/fr-fr/services/container-instances/

## 5.1 Définition ACI

In [20]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Azure ML ACI'}, 
    description = 'Petit exemple')

## 5.2 Génération ACI

In [22]:
# Génération ACI
# Prévoir 7 minutes de temps de traitement

service_name = "monserviceaci"

webservice = Webservice.deploy_from_model(
  workspace=ws, 
  name=service_name, 
  deployment_config=aci_config,
  models = [registered_model], 
  image_config=image_config, 
  )

webservice.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image monserviceaci:1, operation "Succeeded"
Creating service
Running...................
SucceededACI service creation operation finished, operation "Succeeded"

## 5.3 Test du service web ACI

In [24]:
import json
age = 55
km = 4000

test_data  = json.dumps([[age,km]])

print("- Exemple 1 :")
print("Données à scorer :", test_data)
result = webservice.run(input_data=test_data)
print("Scoring = ", result)

age = 25
km = 40000

test_data  = json.dumps([[age,km]])
print()
print("- Exemple 2 :")
print("Données à scorer :", test_data)
result = webservice.run(input_data=test_data)
print("Scoring = ", result)

- Exemple 1 :
Données à scorer : [[55, 4000]]
Scoring = [1]

- Exemple 2 :
Données à scorer : [[25, 40000]]
Scoring = [0]

## 6. Déploiement de l'image containe dans Azure Kubernetes Service (AKS)

## 6.1 Définition AKS

Azure Kubernetes Service (AKS) gère votre environnement Kubernetes hébergé, accélérant et facilitant ainsi le déploiement et la gestion des applications conteneurisées sans expertise d’orchestration de conteneur. Il élimine également la charge des opérations en cours et la maintenance par configuration, la mise à niveau et la mise à l’échelle des ressources à la demande, sans déconnecter vos applications.
> https://azure.microsoft.com/fr-fr/services/kubernetes-service/

In [28]:
# Provisionnement AKS
# Prévoir 10 minutes de temps de traitements si pas déjà provisionné

from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

# Use the default configuration, overriding the default location to a known region that supports AKS
prov_config = AksCompute.provisioning_configuration(location='westus2')

aks_name = 'aks-cluster' 

# Création du cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

aks_target.wait_for_completion(show_output = True)
print("Statut :")
print(aks_target.provisioning_state)

print()
print("Erreur de provisonnement :")
print(aks_target.provisioning_errors)

SucceededProvisioning operation finished, operation "Succeeded"
Statut :
Succeeded

Erreur de provisonnement :
None

## 6.2 Déploiement AKS

In [30]:
# Déploiement AKS (prévoir 7 minutes de temps de traitement environ)

aks_config = AksWebservice.deploy_configuration()

aks_service_name ='monmodelevoitureaks'

aks_service = Webservice.deploy_from_model(
  workspace=ws, 
  name=aks_service_name, 
  deployment_config=aks_config,
  models = [registered_model], 
  image_config=image_config,
  deployment_target=aks_target
  )

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating image
Image creation operation finished for image monmodelevoitureaks:1, operation "Succeeded"
Creating service
Running.....................
SucceededAKS service creation operation finished, operation "Succeeded"
Healthy

## 6.3 Test du modèle déployé dans AKS

In [32]:
# Test du webservice dans AKS

import json
age = 52
km = 10000
test_data  = json.dumps([[age,km]])

print("- Premier exemple :")
print("Données à scorer : ", test_data)
result = aks_service.run(input_data=test_data)
print("Réponse du modèle : ", result)

print()

print("- Second exemple :")
age = 25
km = 40000
test_data  = json.dumps([[age,km]])

print("Données à scorer : ", test_data)
result = aks_service.run(input_data=test_data)
print("Réponse du modèle : ", result)

(- Premier exemple :
Données à scorer : [[52, 10000]]
Réponse du modèle : [1]

- Second exemple :
Données à scorer : [[25, 40000]]
Réponse du modèle : [0]

In [33]:
# Suppression
aks_service.delete()
#image.delete()
model.delete()
aks_target.delete() 

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-355110572855099> in <module> () 
 1 # Suppression 
 2 aks_service . delete ( ) 
 ----> 3 image . delete ( ) 
 4 model . delete ( ) 
 5 aks_target . delete ( ) 

 NameError : name 'image' is not defined

> Fin